In [57]:
from dotenv import load_dotenv
import os

load_dotenv()
OPEN_API_KEY = os.environ.get("OPEN_API_KEY")
HUGGINGFACEHUB_API_TOKEN = os.environ.get("HUGGINGFACEHUB_API_TOKEN")

In [81]:
from langchain.llms import OpenAI

In [9]:
os.environ["OPEN_API_KEY"] = ""

In [11]:
llm = OpenAI(openai_api_key=os.environ["OPEN_API_KEY"], temperature=0.6)

### OpenAI
#### temperature: 
- how creative we want our model to be
- ranges from 0~1 the more closer to 1, the more creative

In [ ]:
text = "What is the capital of Korea"

print(llm.predict(text))

In [12]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""


In [31]:
from langchain import HuggingFaceHub
llm_hugginface = HuggingFaceHub(repo_id="google/flan-t5-large", model_kwargs={"temperature":0, "max_length":256})

In [27]:
output = llm_hugginface.predict("Can you tell me the capital of Korea in full sentence")
output

'Seoul'

In [33]:
output = llm_hugginface.predict("Can you write a poem about AI")
output

'i love the way i look at the world i love the way i feel'

## Promt Templates

In [35]:
from langchain.prompts import PromptTemplate

prompt_template =PromptTemplate(
    input_variables=['country'],
    template="Tell me the capital of {country}"
    )

prompt_template.format(country="United States")

'Tell me the capital of United States'

In [40]:
from langchain.chains import LLMChain
# Chain combines templates with input and executes

chain = LLMChain(llm=llm_hugginface, prompt=prompt_template)
chain.run("Japan")

'kyoto'

## Combining Multiple Chains Using simpoe Sequential Chain

In [46]:
capital_prompt = PromptTemplate(input_variables=['country'], 
                                template="Please tell me the capital of the {country}")

capital_chain = LLMChain(llm=llm_hugginface, prompt=capital_prompt)

famous_template = PromptTemplate(input_variables=['capital'],
                                 template="Suggest me some amzaing places to visin in {capital}")

In [47]:
famous_chain = LLMChain(llm=llm_hugginface, prompt=famous_template)

In [49]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains = [capital_chain, famous_chain])
chain.run("India")

'The Thrissur Museum'

## Sequential Chain

show all outputs from each chain

In [50]:

capital_prompt = PromptTemplate(input_variables=['country'], 
                                template="Please tell me the capital of the {country}")

capital_chain = LLMChain(llm=llm_hugginface, prompt=capital_prompt, output_key="capital")

In [51]:
famous_template = PromptTemplate(input_variables=['capital'],
                                 template="Suggest me some amzaing places to visin in {capital}")

famous_chain = LLMChain(llm=llm_hugginface, prompt=famous_template, output_key='places')

In [52]:
from langchain.chains import SequentialChain

chain = SequentialChain(chains=[capital_chain, famous_chain],
                        input_variables=['country'],
                        output_variables=['capital', 'places'])

In [55]:
chain({'country':"Korea"})

{'country': 'Korea', 'capital': 'seoul', 'places': 'Seoul National Museum'}

## Chatmodels With ChatOpenAI

In [64]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [70]:
chatllm = ChatOpenAI(openai_api_key = OPEN_API_KEY, temperature=0.6, model='gpt-3.5-turbo')

In [72]:
chatllm([
    SystemMessage(content="You're a comedian AI assistant"),
    HumanMessage(content="Provide me some comedy puchlines on AI")
])

# returns AIMessage with multiple punchlines
# AIMesage(content='1. "AI may be smart, but can it tell me if my outfit makes me look like a potato")


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

## Promtp Template + LLM + Output Parsers

In [73]:
from langchain.chat_models import ChatOllama
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [74]:
class Commaseperatedoutput(BaseOutputParser):
    def parse(self, text:str):
        return text.strip().split(",")
    
    

In [78]:
template = "You're a helpful assistant. When the user gives any input, you should generate 5 synonyms words in a comma seperated list"
human_template = "{text}"
chatprompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template)
])

In [79]:
chain = chatprompt|chatllm|Commaseperatedoutput()

In [80]:
chain.invoke({"text":"intelligent"})

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}